<a href="https://colab.research.google.com/github/aniketSanyal/DifferentialPrivacy/blob/main/Last_Layer_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch

In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")


from transformers import AutoTokenizer

checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


class Config:
  num_train_epochs = 7
  learning_rate = 1e-3
  n_prompt_tokens = 10
  random_range  = 0.5
  batch_size = 256
  max_grad_norm = 0.1
args = Config()


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["sentence"], max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")



from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True, padding_value=0)
    labels = torch.stack([item['labels'] for item in batch])

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)


from transformers import AutoModelForSequenceClassification



import torch
import torch.nn as nn
import torch.optim as optim


def get_new_soft_prompt_model(num_labels):
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)
  return model






Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

import numpy as np

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss, logits = outputs[:2]

        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        labels =batch['labels'].detach().cpu().numpy()

        loss_arr.append(loss.item())
        accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [11]:
model =get_new_soft_prompt_model(2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
trainable_params = 0
for i, p in enumerate(model.parameters()):
  if i==40  or i==39:
    trainable_params += p.numel()
    p.requires_grad =True
  else:
    p.requires_grad = False


In [9]:
DELTA = 1 / len(train_dataloader)

LOGGING_INTERVAL = 100

In [14]:

from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


In [19]:
def train_last_layer(model,optimiser, train_dataloader):
  for epoch in range(1, args.num_train_epochs+1):
    losses = []

    for  step, batch in enumerate(train_dataloader):
            optimiser.zero_grad()
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # output = loss, logits, hidden_states, attentions

            loss = outputs[0]

            loss.backward()
            losses.append(loss.item())
            optimiser.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)
                eps = privacy_engine.get_epsilon(DELTA)
                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                  f"ɛ: {eps:.2f}"
                )

In [20]:
train_last_layer(model, optimiser, train_dataloader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1324: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value enco

Epoch: 1 | Step: 100 | Train loss: 2.122 | Eval loss: 2.395 | Eval accuracy: 0.511 | ɛ: 2.64
Epoch: 1 | Step: 200 | Train loss: 2.086 | Eval loss: 2.307 | Eval accuracy: 0.511 | ɛ: 3.29
Epoch: 2 | Step: 100 | Train loss: 1.878 | Eval loss: 2.050 | Eval accuracy: 0.511 | ɛ: 4.04
Epoch: 2 | Step: 200 | Train loss: 1.825 | Eval loss: 1.933 | Eval accuracy: 0.511 | ɛ: 4.42
Epoch: 3 | Step: 100 | Train loss: 1.588 | Eval loss: 1.685 | Eval accuracy: 0.512 | ɛ: 4.97
Epoch: 3 | Step: 200 | Train loss: 1.544 | Eval loss: 1.491 | Eval accuracy: 0.516 | ɛ: 5.28
Epoch: 4 | Step: 100 | Train loss: 1.383 | Eval loss: 1.372 | Eval accuracy: 0.544 | ɛ: 5.75
Epoch: 4 | Step: 200 | Train loss: 1.354 | Eval loss: 1.318 | Eval accuracy: 0.560 | ɛ: 6.02
Epoch: 5 | Step: 100 | Train loss: 1.252 | Eval loss: 1.119 | Eval accuracy: 0.598 | ɛ: 6.45
Epoch: 5 | Step: 200 | Train loss: 1.206 | Eval loss: 1.110 | Eval accuracy: 0.601 | ɛ: 6.70
Epoch: 6 | Step: 100 | Train loss: 1.212 | Eval loss: 1.132 | Eval acc

QNLI

In [22]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


raw_datasets = load_dataset("glue", "qnli")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question"],example["sentence"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'sentence', 'question'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

model =get_new_soft_prompt_model(2)
optimiser = torch.optim.AdamW(model.parameters(), lr=args.learning_rate)

for i, p in enumerate(model.parameters()):
  if i==40  or i==39:
    p.requires_grad =True
  else:
    p.requires_grad = False
model.train()
privacy_engine = PrivacyEngine()

model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)

train_last_layer(model, optimiser, train_dataloader)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1 | Step: 100 | Train loss: 0.753 | Eval loss: 0.719 | Eval accuracy: 0.544 | ɛ: 2.70
Epoch: 2 | Step: 100 | Train loss: 1.039 | Eval loss: 0.854 | Eval accuracy: 0.554 | ɛ: 4.05
Epoch: 3 | Step: 100 | Train loss: 1.057 | Eval loss: 1.021 | Eval accuracy: 0.564 | ɛ: 4.98
Epoch: 4 | Step: 100 | Train loss: 1.226 | Eval loss: 1.079 | Eval accuracy: 0.561 | ɛ: 5.76
Epoch: 5 | Step: 100 | Train loss: 1.241 | Eval loss: 1.117 | Eval accuracy: 0.565 | ɛ: 6.46
Epoch: 6 | Step: 100 | Train loss: 1.270 | Eval loss: 1.160 | Eval accuracy: 0.572 | ɛ: 7.11
Epoch: 7 | Step: 100 | Train loss: 1.266 | Eval loss: 1.159 | Eval accuracy: 0.570 | ɛ: 7.71


QQP

In [23]:
raw_datasets = load_dataset("glue", "qqp")


tokenizer = AutoTokenizer.from_pretrained(checkpoint)


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])
raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])
raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question1"],example["question2"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'question1', 'question2'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)


model =get_new_soft_prompt_model(2)
optimiser = torch.optim.AdamW(model.parameters(), lr=args.learning_rate)



Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
for i, p in enumerate(model.parameters()):
  if i==40  or i==39:
    p.requires_grad =True
  else:
    p.requires_grad = False



In [26]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


train_last_layer(model, optimiser, train_dataloader)

/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_f

Epoch: 1 | Step: 100 | Train loss: 1.709 | Eval loss: 2.031 | Eval accuracy: 0.639 | ɛ: 2.70
Epoch: 2 | Step: 100 | Train loss: 2.003 | Eval loss: 1.904 | Eval accuracy: 0.639 | ɛ: 4.05
Epoch: 3 | Step: 100 | Train loss: 1.982 | Eval loss: 1.976 | Eval accuracy: 0.639 | ɛ: 4.98
Epoch: 4 | Step: 100 | Train loss: 1.995 | Eval loss: 1.955 | Eval accuracy: 0.639 | ɛ: 5.76
Epoch: 5 | Step: 100 | Train loss: 1.973 | Eval loss: 1.988 | Eval accuracy: 0.639 | ɛ: 6.46
Epoch: 6 | Step: 100 | Train loss: 1.971 | Eval loss: 1.944 | Eval accuracy: 0.639 | ɛ: 7.11
Epoch: 7 | Step: 100 | Train loss: 1.971 | Eval loss: 1.917 | Eval accuracy: 0.639 | ɛ: 7.71


MNLI

In [29]:
raw_datasets = load_dataset("glue", "mnli")


from datasets import load_metric
metric = load_metric("glue", "mnli")

checkpoint ="prajjwal1/bert-tiny-mnli"

raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation_matched'] = raw_datasets['validation_matched'].select([i for i in range(5000)])




tokenizer = AutoTokenizer.from_pretrained(checkpoint, num_classes =3)


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["premise"],example["hypothesis"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'premise', 'hypothesis'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation_matched"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)


model =AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
optimiser = torch.optim.AdamW(model.parameters(), lr=args.learning_rate)
for i, p in enumerate(model.parameters()):
  if i==40  or i==39:
    p.requires_grad =True
  else:
    p.requires_grad = False

model.train()
privacy_engine = PrivacyEngine()

model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


train_last_layer(model, optimiser, train_dataloader)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_f

Epoch: 1 | Step: 100 | Train loss: 1.106 | Eval loss: 1.079 | Eval accuracy: 0.390 | ɛ: 2.70
Epoch: 2 | Step: 100 | Train loss: 1.160 | Eval loss: 1.171 | Eval accuracy: 0.403 | ɛ: 4.05
Epoch: 3 | Step: 100 | Train loss: 1.233 | Eval loss: 1.302 | Eval accuracy: 0.400 | ɛ: 4.98
Epoch: 4 | Step: 100 | Train loss: 1.266 | Eval loss: 1.332 | Eval accuracy: 0.402 | ɛ: 5.76
Epoch: 5 | Step: 100 | Train loss: 1.258 | Eval loss: 1.347 | Eval accuracy: 0.397 | ɛ: 6.46
Epoch: 6 | Step: 100 | Train loss: 1.278 | Eval loss: 1.247 | Eval accuracy: 0.417 | ɛ: 7.11
Epoch: 7 | Step: 100 | Train loss: 1.294 | Eval loss: 1.313 | Eval accuracy: 0.404 | ɛ: 7.71


Last Layer non private fine tuning for MNLI dataset

In [33]:
model =AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
optimiser = torch.optim.AdamW(model.parameters(), lr=args.learning_rate)
for i, p in enumerate(model.parameters()):
  if i==40  or i==39:
    p.requires_grad =True
  else:
    p.requires_grad = False

In [34]:
def train_last_layer(model,optimiser, train_dataloader):
  for epoch in range(1, args.num_train_epochs+1):
    losses = []

    for  step, batch in enumerate(train_dataloader):
            optimiser.zero_grad()
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # output = loss, logits, hidden_states, attentions

            loss = outputs[0]

            loss.backward()
            losses.append(loss.item())
            optimiser.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)
                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                )

In [35]:
train_last_layer(model, optimiser, train_dataloader)

Epoch: 1 | Step: 100 | Train loss: 1.081 | Eval loss: 1.054 | Eval accuracy: 0.433 | 
Epoch: 2 | Step: 100 | Train loss: 1.070 | Eval loss: 1.055 | Eval accuracy: 0.435 | 
Epoch: 3 | Step: 100 | Train loss: 1.067 | Eval loss: 1.058 | Eval accuracy: 0.426 | 
Epoch: 4 | Step: 100 | Train loss: 1.063 | Eval loss: 1.054 | Eval accuracy: 0.438 | 
Epoch: 5 | Step: 100 | Train loss: 1.066 | Eval loss: 1.061 | Eval accuracy: 0.420 | 
Epoch: 6 | Step: 100 | Train loss: 1.063 | Eval loss: 1.054 | Eval accuracy: 0.438 | 
Epoch: 7 | Step: 100 | Train loss: 1.063 | Eval loss: 1.063 | Eval accuracy: 0.416 | 
